In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import random
from itertools import combinations
import re
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModel
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, precision_score, recall_score


/kaggle/input/tweet-pairs/tp.csv


# Data Preparation

In [ ]:
train = pd.read_excel(r"C:\Users\islam\Desktop\MASTER 1 s2\NLP\PR1\train.xlsx")
test = pd.read_excel(r"C:\Users\islam\Desktop\MASTER 1 s2\NLP\PR1\test.xlsx")

In [ ]:
#we shrink the data so we can make combinations in our machine 
train_dropped = train.groupby('user').sample(frac=0.5, random_state=42)
len(train_dropped)

13000

In [ ]:
import random 
# Create an empty dataframe to store the pairs
pairs_df = pd.DataFrame(columns=['tweet1', 'tweet2', 'label'])

# Iterate over each unique user in train_dropped
for user in train_dropped['user'].unique():
    # Get all tweets from the current user
    user_tweets = train_dropped[train_dropped['user'] == user]['text']
    
    # Generate all possible combinations of tweet pairs
    tweet_pairs = list(combinations(user_tweets, 2))
    
    # Randomly sample an equal number of positive and negative pairs
    num_pairs = min(len(tweet_pairs), len(train_dropped[train_dropped['user'] != user]))
    positive_pairs = random.sample(tweet_pairs, num_pairs)
    negative_pairs = random.sample(list(combinations(train_dropped[train_dropped['user'] != user]['text'], 2)), num_pairs)
    
    # Add the positive pairs with label 1
    pairs_df = pairs_df.append(pd.DataFrame({'tweet1': [pair[0] for pair in positive_pairs],
                                             'tweet2': [pair[1] for pair in positive_pairs],
                                             'label': [1] * num_pairs}), ignore_index=True)
    
    # Add the negative pairs with label 0
    pairs_df = pairs_df.append(pd.DataFrame({'tweet1': [pair[0] for pair in negative_pairs],
                                             'tweet2': [pair[1] for pair in negative_pairs],
                                             'label': [0] * num_pairs}), ignore_index=True)

# Shuffle the dataframe
pairs_df = pairs_df.sample(frac=1).reset_index(drop=True)

# Print the resulting dataframe
pairs_df


C:\Users\islam\AppData\Local\Temp\ipykernel_14472\4279469501.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  pairs_df = pairs_df.append(pd.DataFrame({'tweet1': [pair[0] for pair in positive_pairs],
C:\Users\islam\AppData\Local\Temp\ipykernel_14472\4279469501.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  pairs_df = pairs_df.append(pd.DataFrame({'tweet1': [pair[0] for pair in negative_pairs],


,tweet1,tweet2,label
0,#DWTS tonight on @abc at 8,NYC! How I miss you! http://www.thecobrasnake....,0
1,"MEET THE QUEEN OF MEMPHIS (Egypt, that is): ht...",rephrase: IS everything!,1
2,"Out of our very big country, with many choices...",Thank you to Brad Blakeman on @FoxNews for gra...,1
3,http://twitpic.com/ddm9z - Just showed 2 my ho...,@ravensuxx YAS ya herd,1
4,I urge all Americans to participate in #takeba...,We now have confirmation as to one reason Croo...,1
...,...,...,...
311995,".@BethanyMota’s known to let the beat … mmm, d...",Vengeance is SWEET! https://goo.gl/2UwF3s pic....,1
311996,Broken hearted for the families tonight. Broke...,"and THANK YOU... by the way, it's all because ...",1
311997,23 points\n20 assists \n9 rebounds \n@russwest...,Vote for who YOU think should be the #KiaMVP &...,1
311998,pic.twitter.com/60uQL39qpX,@enigmandaa NO SLEEP TILL ARTPOP IS FINISHED. ...,0


In [ ]:
import random 
# Create an empty dataframe to store the pairs
test_pairs_df = pd.DataFrame(columns=['tweet1', 'tweet2', 'label'])

# Iterate over each unique user in train_dropped
for user in test['user'].unique():
    # Get all tweets from the current user
    user_tweets = test[test['user'] == user]['text']
    
    # Generate all possible combinations of tweet pairs
    tweet_pairs = list(combinations(user_tweets, 2))
    
    # Randomly sample an equal number of positive and negative pairs
    num_pairs = min(len(tweet_pairs), len(test[test['user'] != user]))
    positive_pairs = random.sample(tweet_pairs, num_pairs)
    negative_pairs = random.sample(list(combinations(test[test['user'] != user]['text'], 2)), num_pairs)
    
    # Add the positive pairs with label 1
    test_pairs_df = test_pairs_df.append(pd.DataFrame({'tweet1': [pair[0] for pair in positive_pairs],
                                             'tweet2': [pair[1] for pair in positive_pairs],
                                             'label': [1] * num_pairs}), ignore_index=True)
    
    # Add the negative pairs with label 0
    test_pairs_df = test_pairs_df.append(pd.DataFrame({'tweet1': [pair[0] for pair in negative_pairs],
                                             'tweet2': [pair[1] for pair in negative_pairs],
                                             'label': [0] * num_pairs}), ignore_index=True)

# Reset the index of the dataframe
test_pairs_df = test_pairs_df.reset_index(drop=True)

# Shuffle the dataframe
test_pairs_df = test_pairs_df.sample(frac=1).reset_index(drop=True)

# Print the resulting dataframe
test_pairs_df


C:\Users\islam\AppData\Local\Temp\ipykernel_14472\958875753.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  test_pairs_df = test_pairs_df.append(pd.DataFrame({'tweet1': [pair[0] for pair in positive_pairs],
C:\Users\islam\AppData\Local\Temp\ipykernel_14472\958875753.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  test_pairs_df = test_pairs_df.append(pd.DataFrame({'tweet1': [pair[0] for pair in negative_pairs],


,tweet1,tweet2,label
0,Interview in Nature describing why we should e...,Latest S and X tires have internal acoustic foam,1
1,Me @tarasavelo & @fredericaspiras want to than...,pic.twitter.com/8kZtM1pd,0
2,so many activities it is making my head spin! ...,Will be participating in a town hall event- ho...,0
3,Thanks for stopping by!,Nah,1
4,"Oh hi there, beautiful people \n\nGreetings A...",RT if you agree: Romney must withdraw his supp...,0
...,...,...,...
31195,This hug could be yours when you go backstage ...,FACT: President Obama’s plan will help create ...,1
31196,This is and will always be who I truly am in m...,"""All of us have a right to be proud of the pro...",0
31197,she bored http://instagram.com/p/w6zhIYBM_H/,SECOND HALF Who will find the net in the secon...,0
31198,"By the end of March, 1.8 million people had do...",President Obama: Colleges that keep tuition do...,1


In [ ]:
pairs_df.to_excel(r"C:\Users\islam\Desktop\MASTER 1 s2\NLP\PR1\tp.xlsx", index=False)
test_pairs_df.to_excel(r"C:\Users\islam\Desktop\MASTER 1 s2\NLP\PR1\tp_test.xlsx", index=False)



# Data Preprocessing

In [2]:
# we did this part in kaggle where we moved uploaded 
dataset = pd.read_csv("/kaggle/input/tweet-pairs/tp.csv") 
dataset_test = pd.read_csv("/kaggle/input/tweet-pairs/tp_test.csv") 

In [3]:
dataset['tweet1'] = dataset['tweet1'].apply(lambda x: re.sub(r'[^a-zA-Z0-9@# ]', '', x))
dataset['tweet2'] = dataset['tweet2'].apply(lambda x: re.sub(r'[^a-zA-Z0-9@# ]', '', x))

In [4]:
dataset['tweet1'] = dataset['tweet1'].str.lower()
dataset['tweet2'] = dataset['tweet2'].str.lower()

In [5]:
# Sample 30,000 rows randomly
train_data = dataset.sample(n=30000, random_state=42)
train_data

,tweet1,tweet2,label
240080,you asked greetings from holland httpbitly1l...,i like @youshidean how do u like this pic i re...,1
39292,good news is melanias speech got more publicit...,james comey leaked classified information to t...,1
11310,ontem conseguirmos uma importante vitria para ...,@jessieriri well never let you down @rihanna ...,0
107379,any senator who votes against starting debate ...,pennsylvania poll just released two rallies th...,1
218157,@gavinfree @danielgruchy once again thats the ...,cuz i love love cuz i love love you better tha...,0
...,...,...,...
206591,thank you for your support on my way now see ...,it now turns out that the phony allegations ag...,1
304943,shadowy conservative groups have pledged 1 bil...,fact obamas clean fuel standards would cut the...,1
253756,this is your chance to meet president obama no...,president obama if theres even one thing we ca...,1
296232,this saturday @realmadrid vs @lagalaxy follow ...,dog snuggle thread on reddit comments get real...,0


In [ ]:
test_data = dataset_test.sample(n=10000, random_state=42)
test_data

In [7]:
train_data1 = train_data
train_data = train_data.head(10000)


In [8]:
train_data

,tweet1,tweet2,label
13691,2012 battleground states director mitch stewar...,this month and every month #loveislovepictwitt...,1
93939,casually dropped a couple more 50mp wallpaperw...,ingles slick nolook behindtheback dish is toni...,0
84163,trading buckets on @nbaontnt#doitbig#sacrament...,its in your handsvote for your favorite moment...,1
8859,hilariousridiculous @youtube bug of the day de...,stock market could hit alltime high again 2200...,0
48498,add your name to help millions of americans wh...,the united states must greatly strengthen and ...,0
...,...,...,...
245651,amazing day just being creative in the studio ...,@perezhilton crossing my finger for you on wed...,0
244699,@chiefkeef this is the best thanksgiving ever ...,my fans make me so happy she was waiting outsi...,0
113464,happy birthday @rebelwilson love the other hal...,please remind me to never ask @diddy to do bab...,1
139474,baltimore stand up hope yall ready to get on #...,flying to #loudtour rehearsal,1


In [30]:
test_data1 = test_data
test_data = test_data.head(3000)
test_data


,tweet1,tweet2,label
30818,whatever you call them there were some specia...,goal southampton 03 leicester 38 minswhat a ha...,1
210944,#rihannabetawards,im blessed to have you guys man yall some diff...,0
192438,#clearmenpictwittercomubo0khu2go,thinking of all the students and families of u...,0
1965,@asyouread14 some timesdays are just more conv...,@missjillscott @rihanna following me make my h...,1
230198,working out to recover for the next challenge ...,chose my dress for the day with a big thumbs u...,0
...,...,...,...
41427,this,that is terrible lol,1
168787,special guest today in training ground pictwit...,its on hold until next time @mrivera85 @ochoci...,1
39549,someone reading this tweet is going to win all...,applause will be available first on us itunes ...,0
70425,week left #btsarmy were counting down the days...,the devil wants to steal your joy because he k...,0


In [11]:
class BertModel(nn.Module):
    def __init__(self, transformer_model):
        super(BertModel, self).__init__()
        self.transformer = transformer_model
        self.fc = nn.Linear(768, 1)  
        self.fc2 = nn.Linear(1,1)

    def forward(self, input_ids1, attention_mask1,input_ids2, attention_mask2):
        output1 = self.transformer(input_ids1, attention_mask1)[0] 
        output2 = self.transformer(input_ids2, attention_mask2)[0]
        output1 = torch.mean(output1, dim=1) 
        output2 = torch.mean(output2, dim=1) 
        output1 = self.fc(output1)
        output2 = self.fc(output2)
        output1 = torch.sigmoid(output1)
        output2 = torch.sigmoid(output2)
        distance = torch.abs(output1 - output2)
        ret = self.fc2(distance)        
        return  torch.sigmoid(ret)

In [12]:
# tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
# model = AutoModel.from_pretrained("distilbert-base-uncased")

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
model = AutoModel.from_pretrained("bert-base-uncased")


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = BertModel(model).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)
criterion = nn.BCELoss()

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [13]:
# Training loop
num_epochs = 2
for epoch in range(num_epochs):
    model.train() 
    print("here")
    for index, row in train_data.iterrows():
        
        input_ids1 = tokenizer(row['tweet1'], return_tensors='pt', padding=True, truncation=True)['input_ids'].to(device)
        input_ids2 = tokenizer(row['tweet2'], return_tensors='pt', padding=True, truncation=True)['input_ids'].to(device)
        attention_mask1 = tokenizer(row['tweet1'], return_tensors='pt', padding=True, truncation=True)['attention_mask'].to(device)
        attention_mask2 = tokenizer(row['tweet2'], return_tensors='pt', padding=True, truncation=True)['attention_mask'].to(device)
        label = torch.tensor(row['label'], dtype=torch.float).to(device)
      
        optimizer.zero_grad()
        model_output = model( input_ids1, attention_mask1,input_ids2, attention_mask2)
        
        loss = criterion(model_output.squeeze(), label)
        loss.backward()
        optimizer.step()
        print("loss",loss," epouch ",f'Epoch [{epoch+1}/{num_epochs}]')
print("end")
  

here
loss tensor(0.7683, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward0>)  epouch  Epoch [1/2]
loss tensor(0.6295, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward0>)  epouch  Epoch [1/2]
loss tensor(0.7666, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward0>)  epouch  Epoch [1/2]
loss tensor(0.6283, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward0>)  epouch  Epoch [1/2]
loss tensor(0.6217, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward0>)  epouch  Epoch [1/2]
loss tensor(0.7591, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward0>)  epouch  Epoch [1/2]
loss tensor(0.6220, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward0>)  epouch  Epoch [1/2]
loss tensor(0.6237, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward0>)  epouch  Epoch [1/2]
loss tensor(0.7688, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward0>)  epouch  Epoch [1/2]
loss tensor(0.6201, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward0>)  epouch  Epoch [1/2]
loss tensor(0.7368, devic

In [31]:
model.eval()

predicted_labels = []
true_labels = []
with torch.no_grad():
    for index, row in test_data.iterrows():
        input_ids1 = tokenizer(row['tweet1'], return_tensors='pt', padding=True, truncation=True)['input_ids'].to(device)
        input_ids2 = tokenizer(row['tweet2'], return_tensors='pt', padding=True, truncation=True)['input_ids'].to(device)
        attention_mask1 = tokenizer(row['tweet1'], return_tensors='pt', padding=True, truncation=True)['attention_mask'].to(device)
        attention_mask2 = tokenizer(row['tweet2'], return_tensors='pt', padding=True, truncation=True)['attention_mask'].to(device)
        label = torch.tensor(row['label'], dtype=torch.float).to(device)



        model_output = model( input_ids1, attention_mask1,input_ids2, attention_mask2)
        print("model_output ",similarity_score)
        predicted_label = torch.round(model_output.squeeze()).cpu().numpy()
        predicted_labels.append(predicted_label)
        true_labels.append(row['label'])

model_output  tensor([[0.4636]], device='cuda:0')
model_output  tensor([[0.4636]], device='cuda:0')
model_output  tensor([[0.4636]], device='cuda:0')
model_output  tensor([[0.4636]], device='cuda:0')
model_output  tensor([[0.4636]], device='cuda:0')
model_output  tensor([[0.4636]], device='cuda:0')
model_output  tensor([[0.4636]], device='cuda:0')
model_output  tensor([[0.4636]], device='cuda:0')
model_output  tensor([[0.4636]], device='cuda:0')
model_output  tensor([[0.4636]], device='cuda:0')
model_output  tensor([[0.4636]], device='cuda:0')
model_output  tensor([[0.4636]], device='cuda:0')
model_output  tensor([[0.4636]], device='cuda:0')
model_output  tensor([[0.4636]], device='cuda:0')
model_output  tensor([[0.4636]], device='cuda:0')
model_output  tensor([[0.4636]], device='cuda:0')
model_output  tensor([[0.4636]], device='cuda:0')
model_output  tensor([[0.4636]], device='cuda:0')
model_output  tensor([[0.4636]], device='cuda:0')
model_output  tensor([[0.4636]], device='cuda:0')


In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

test_labels = test_data['label'].tolist()

# Assuming test_labels contains the actual labels for your test data
accuracy = accuracy_score(test_labels, predictions)
precision = precision_score(test_labels, predictions)
recall = recall_score(test_labels, predictions)
f1 = f1_score(test_labels, predictions)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)

#precision: 0.47 f1_score :1